# Lyric Generator!

Further to my studies of NLP, I decided to see whether I could harness AI to write new lyrics based on my previous listening habits. 

First I pulled my data from the Spotify API. Initially I was going to use only my top 100 songs from 2019 (according to Spotify) but I decided to use the same from 2018 as well in order to increased the size of the corpus used to train my model. I have left the code here for reference, but you can find the data that I actually used here. 

In order to run my notebook, please download the dataset and put it in the data folder. 

In [86]:
#Import libraries
import spotipy.util as util
import spotipy
import os.path as path
import requests
from bs4 import BeautifulSoup
import re
from spotipy.oauth2 import SpotifyClientCredentials
from better_profanity import profanity
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense, Input, Embedding, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np
import random

In [87]:

username = 'ninahew'
spotify_client_id ='a758a8092e3b45949388c4123f79948f'
spotify_client_secret = 'df4c710d84b64ab7810f0b9b055a07f2'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'
genius_key = 'sQkNdeBg0aT9rz89yElRfHbxE_UIW3kFs9IkXPIH6Ab4cH8DvfikT09FU56MVjZP'
spotify_playlist_id = 'spotify:playlist:37i9dQZF1EtmAVggpyt82c'
spotify_user_id = 'spotify:user:ninahew'


# spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, 
#                                                                               client_secret=client_secret))

In order to pull the lyrics to my playlists, there are a few steps that need to be done.
From Spotify:
1) Pull the playlist info

2) Pull the track names for each song

3) Pull the artists names for each song


THEN

4) Using the Genius API, the song name and artist, pull the song information

5) Check whether the tracks are included on the Genius API

6) If they're included, pull the url for the song's page

7) Scrape the lyrics from the page

8) Finally, return the results

In [88]:

# class GetLyrics():
    
#     def __init__(self, spotify_client_id, spotify_client_secret, user_id, playlist_id, genius_key):
#         self.spotify_client_id = spotify_client_id
#         self.spotify_client_secret = spotify_client_secret
#         self.user_id = user_id
#         self.playlist_id = playlist_id
#         self.genius_key = genius_key
        
#     def get_playlist_info(self):
#         token = SpotifyClientCredentials(client_id=self.spotify_client_id, client_secret=self.spotify_client_secret).get_access_token()
#         sp = spotipy.Spotify(token)
#         playlist = sp.user_playlist_tracks(self.user_id, self.playlist_id)
#         self.playlist = playlist
#         return self.playlist
    
#     def get_track_names(self):
#         track_names = []
#         for song in range(len(self.playlist['items'])):
#             track_names.append(self.playlist['items'][song]['track']['name'])
#         self.track_names = track_names
#         return self.track_names
    
#     def get_track_artists(self):
#         track_artists = []
#         for song in range(len(self.playlist['items'])):
#             track_artists.append(self.playlist['items'][song]['track']['artists'][0]['name'])
#         self.track_artists = track_artists
#         return self.track_artists
        
#     def request_song_info(self, track_name, track_artist):
#         self.track_name = track_name
#         self.track_artist = track_artist
#         base_url = 'https://api.genius.com'
#         headers = {'Authorization': 'Bearer ' + self.genius_key}
#         search_url = base_url + '/search'
#         data = {'q': track_name + ' ' + track_artist}
#         response = requests.get(search_url, data=data, headers=headers)
#         self.response = response
#         return self.response

#     def check_hits(self):
#         json = self.response.json()
#         remote_song_info = None
#         for hit in json['response']['hits']:
#             if self.track_artist.lower() in hit['result']['primary_artist']['name'].lower():
#                 remote_song_info = hit
#                 break
#         self.remote_song_info = remote_song_info
#         return self.remote_song_info
    
#     def get_url(self):
#         song_url = self.remote_song_info['result']['url']
#         self.song_url = song_url
#         return self.song_url
    
#     def scrape_lyrics(self):
#         page = requests.get(self.song_url)
#         html = BeautifulSoup(page.text, 'html.parser')
#         lyrics1 = html.find("div", class_="lyrics")
#         lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
#         if lyrics1:
#             lyrics = lyrics1.get_text()
#         elif lyrics2:
#             lyrics = lyrics2.get_text()
#         elif lyrics1 == lyrics2 == None:
#             lyrics = None
#         return lyrics

#     def get_lyrics(self):
#         playlist = GetLyrics.get_playlist_info(self)
#         track_names = GetLyrics.get_track_names(self)
#         track_artists = GetLyrics.get_track_artists(self)
#         song_lyrics = []
#         for i in range(len(self.track_names)):
#             print("\n")
#             print(f"Working on track {i}.")
#             response = GetLyrics.request_song_info(self, self.track_names[i], self.track_artists[i])
#             remote_song_info = GetLyrics.check_hits(self)
#             if remote_song_info == None:
#                 lyrics = None
#                 print(f"Track {i} is not in the Genius database.")
#             else:
#                 url = GetLyrics.get_url(self)
#                 lyrics = GetLyrics.scrape_lyrics(self)
#                 if lyrics == None:
#                     print(f"Track {i} is not in the Genius database.")
#                 else:
#                     print(f"Retrieved track {i} lyrics!")
#             song_lyrics.append(lyrics)
#         return song_lyrics


Next, I put this into action, pulling both playlists separately. 

In [89]:
# songs2019 = GetLyrics(spotify_client_id, spotify_client_secret, spotify_user_id, spotify_playlist_id, genius_key)
# song_lyrics2019 = songs2019.get_lyrics()

In [90]:
# spotify_playlist_id = 'spotify:playlist:37i9dQZF1Ejy4HBpGZqQ6f'
# songs2018 = GetLyrics(spotify_client_id, spotify_client_secret, spotify_user_id, spotify_playlist_id, genius_key)
# song_lyrics2018 = songs2018.get_lyrics()

I wanted to double check it had worked, so I checked the first two tracks. Turns out those were Juice by Lizzo and Finesse by Bruno Mars. Unexpected, but who am I to argue with the Spotify algorithm.  

In [91]:
# song_lyrics2019[0]

In [92]:
# song_lyrics2018[0]

Combining the two results and checking the size

In [93]:
# song_lyrics = song_lyrics2019 + song_lyrics2018
# len(song_lyrics)

Next, I noticed that I would need to clean up the data a bit. There were a few labels to call out verses/choruses/featured artists, so I wanted to get ride of those. 

In [94]:
# text = song_lyrics[0]
# re.findall('\[.*\]', text)

In [95]:
# section_labels = [re.findall('\[[^\]]+\]', song) for song in song_lyrics if song is not None]

I thought it might be wise to count the instances of these words and take a look at them before I dropped them in case I was getting rid of something that might later become important. 

In [96]:
# label_counter = {}

# for song in section_labels:
#     for label in song:
#         if label not in label_counter:
#             label_counter[label] = 1
#         else:
#             label_counter[label] += 1
# label_counter

Looking at the results, I was comfortable getting rid of these words. Additionally, I eliminated the new lines, and then added spaces in their place. 


Later when I would start generating lyrics, I found that it used profanity quite liberally, so I thought it might be wise to clean up the language a bit to make it less colourful. I used an additional Python library to do this. 

In [97]:
# without_labels = [re.sub('\[[^\]]+\]', '', song) for song in song_lyrics if song is not None]
# without_newline = [re.sub('\n', ' ', song) for song in without_labels]
# respace_words = [re.sub('([a-z])([A-Z])', "\g<1> \g<2>", song) for song in without_newline]
# lyrics_final = [profanity.censor(song) for song in respace_words]

Next, I added the lyrics to a dataframe to make them easier for the algorithm to process. 

In [98]:
lyrics = pd.DataFrame(lyrics_final, columns=['lines'])
lyrics.head()

,lines
0,"Mirror, mirror on the wall Don't say it ’cause..."
1,I've never been closer I tried to understand T...
2,Steppin' out the weekends open wide Fill it ...
3,"I hold you, I touch you In a maze, can't fi..."
4,Have you heard that there's an ad Listed in ...


Finally, the data is ready, so I saved it. In the end, I maanged to pull 137 / 200 songs (the remainder of which weren't in the Genius database). 

In [99]:
# lyrics.to_csv('../data/lyrics', index=False)
lyrics = pd.read_csv('../data/lyrics')

lyrics.shape

(137, 1)

Next, Ive created the x train and y train set. The x train set goes up to the penultimate word in the line, so the target is the final word. 
The y train set starts from the second word in the line to compensate for this. 

In [100]:
# x_train = [line[:-1] for line in lyrics.lines]
# y_train = [line[1:] for line in lyrics.lines]

In order for the model to process the data, it needs numbers instead of words, so I used the tokenizer to assign each word an index. 

In [101]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lyrics.lines)
# x_train = tokenizer.texts_to_sequences(x_train)
# y_train = tokenizer.texts_to_sequences(y_train)

We'll need to switch between the index and words eventually, so the below will assign the 
index and retrieve word from the index later on. 

In [102]:
word2index = tokenizer.word_index
index2word = {value: key for key, value in word2index.items()}

In [103]:
word2index["<pad>"] = 0
index2word[0] = "<pad>"

Finally, I've defined some parameters for the model below using the describe function. 

In [104]:
# lengths = []

# for sequence in x_train:
#     lengths.append(len(sequence))
    
# lengths = pd.Series(lengths)
# lengths.describe()

In [105]:
# max_len = 769
# vocab_size = len(tokenizer.word_index) + 1
# embedding_dim = 128
# vocab_size

In [106]:
# x_train = pad_sequences(x_train, maxlen=max_len, padding='post', truncating='post')
# y_train = pad_sequences(y_train, maxlen=max_len, padding='post', truncating='post')

Next, the model is built. the Sequential function groups a linear stack of layers into a model. We add the GRU which stands for Gated Recurrant Unit, and ask this only to return the last word in the sequence of lyrics rather than the whole line.  The GRU refers to the type of recurrent neural network used here, and the GRU is used instead of a Long short term memory (LSTM) on account of the fact that its faster to learn, but a disadvantage is that they have a shorter term memory for sequences. The model is then optimized to increase speed.


Then, we feed it the data. The first time I did this I used only half the data (before I decided to use 2018 data too), and 50 epochs. This took around half an hour. 

Later on, I retrained with twice the data and 75 epochs - this took 2-3 hours. 

In [107]:
# model = Sequential()
# model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True))
# model.add(GRU(units=1024, return_sequences=True))
# model.add(Dense(vocab_size))

In [108]:
# model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(from_logits=True))

In [109]:
# history = model.fit(x_train, y_train, epochs=75, verbose=1)

To avoid having to retrain the model, you can load the model below. h5 is the first one I trained, h6 is too big to upload to GitHub, so you will have to use h5 instead. 

In [110]:
# model.save("model.h6")
model = load_model("model.h5")
# model = load_model("model.h6")

The function is going to be fed a word, and based on that word, the model will produce a list of indexes (of words) weighted by how likely they are to follow the initial word given the lyrics in the songs we fed it. 


In order that it doesn't loop back on itself, for example:

'I like disco music and I like to dance'

If this was fed to the model without tweaking, from the first part of the sentence you can see that 'disco' is most likely to follow 'like' which is why it was chosen. However, when 'I like' appears again, then the model would want to choose 'disco' again, because that is the most likely word to follow 'like' as we've already seen.

In order to avoid this, the next function will find the top x most likely indices (words) to follow, and will choose a random index (word) from that list instead. 

In [111]:
def select_next_word(predictions):
    '''
    input: list of numbers showing weighting or prediction to be next word, indices correspond to 
    words 
    output: random index from top 5 predicted words 
    '''
    return random.choice(sorted(list(zip(predictions, 
                                         range(len(predictions)))), reverse = True)[:7])[1]
    

Now, for the fun part. The below function calls on the model to generate the new lyrics. Given any word (as long as its in the original corpus) a new sequence will be generated. 

In [112]:

def generate(word):
    inputs = np.zeros((1, 1))
    inputs[0, 0] = word2index[word]
    count = 1
    while count <= 50:
        pred = model.predict(inputs)
        next_word_index = select_next_word(list(pred[0][0]))
        
        inputs[0, 0] = next_word_index
        print(index2word[next_word_index], end=" ")
        count += 1

print(generate('mirror'))


ablaze mine place alone at me the tricks at me magic central is please ever night they'd here wealth need at can yellow funky place night ever is please central it need at me 'round good fever what the ever is please feeling it what get place mine funky hot None


So, as you can see, the lyrics are a little more Bohemian Rhapsody than Bob Dylan, but as a first attempt, I'm pleased with the results. 

To take this further, id be interested to increase the size of the original corpus anad see the impact that this has. 

Likewise, basing the predicted word on only the word before it means that it will be difficult to get a natural, coherent result, and so further investigation for me would be to see whether I can base the prediction on the 2 or 3 words prior. 

I've included some links here to articles which helped me a lot with this project:

https://towardsdatascience.com/become-a-lyrical-genius-4362e7710e43
https://towardsdatascience.com/generating-eminem-lyrics-using-neural-networks-96e7f9c45e8a
